In [ ]:
import pathlib
import re
import collections
import functools
import difflib
import operator
import warnings
import pickle

In [ ]:
%run utils.py
#cache = ApprovedMatches(datadir / "approved.pickle")

# Uhhh, lowercase for realsies

In [ ]:
with AllEntries() as dataSet:
    for fn, year, entries in dataSet.getEntries():
        print(year)
        lines = [commonHeader + "\n"]
        for entry in entries:
            lines.append(entry.toRow().lower() + "\n")
        with fn.open("w", encoding="UTF-8") as fd:
            fd.writelines(lines)

In [ ]:
lines = []
with (datadir / "gps-coords.csv").open("r", encoding="utf-8") as fd:
    header = next(fd)
    for line in fd:
        split = line.split("|")
        split[0] = split[0].lower().strip()
        lines.append("|".join(split))
lines.sort()
lines.insert(0, header)
with (datadir / "gps-coords.csv").open("w", encoding="utf-8") as fd:
    fd.writelines(lines)

# Fix the birthplace field

## Part one: replace "i sognet"

In [ ]:
with AllEntries() as dataSet:
    for fn, year, entries in dataSet.getEntries():
        print(year)
        lines = [commonHeader + "\n"]
        for entry in entries:
            # prøv at fikse fødesteder
            if "her i sognet" in entry.fødested or \
               "heri sognet" in entry.fødested or \
                entry.fødested.replace(".", "") in ("i sognet", "her i s"):        
                # det kan være, der står noget om placering i parentes; brug det
                match1 = re.search(r"(her ?)?i sognet (\{|\[|\()(?P<place>.*?)(\]|\)|\})", entry.fødested)
                # det kan også være, der står noget efter et komma
                match2 = re.search(r"(her ?)? i sognet, (<P<place>.*)", entry.fødested)
                if match1:
                    new = match1.group("place")
                elif match2:
                    new = match2.group("place")
                else:
                    new = entry.sogn
                entry = entry._replace(fødested=new)
            lines.append(entry.toRow() + "\n")
        with fn.open("w", encoding="UTF-8") as fd:
            fd.writelines(lines)

In [ ]:
re.search(r"(her ?)?i sognet (\[|\()(?P<place>.*?)(\]|\))", "her i sognet [tommerup)").group("place")

In [ ]:
# for 1845 som eksempel
sorted((a,b) for (a,b) in changes if a not in ("her i sognet", "heri sognet", "i sognet"))

## Part two: more of the same

Let's see what's missing; the GPS lookup efforts showed a lot of crud.

**TODO**

Change all the simple "dito", "ditto", "do", "do." etc. to the previous birthplace...

In [ ]:
with AllEntries() as dataSet:
    for fn, year, entries in dataSet.getEntries():
        print(year)
        lines = [commonHeader + "\n"]
        prev = None
        for entry in entries:
            place = entry.fødested.replace(".", "")
            if place in ("do", "dito", "ditto"):
                entry = entry._replace(fødested=prev)
            prev = entry.fødested
            lines.append(entry.toRow() + "\n")
        with fn.open("w", encoding="UTF-8") as fd:
            fd.writelines(lines)

In [ ]:
_remove_first = str.maketrans({
    c: "" for c in ",.-:?/!"
})
_remove_last = str.maketrans({
    c: "" for c in "[]{}()"
})

In [ ]:
def extractPlace(name):
    stripped = name.translate(_remove_first)
    match = re.match(r"di?t?t?o(\(|\[)(.*)(\)|\])", stripped.replace(" ", ""))
    if match:
        return match.group(2).translate(_remove_last).strip()
    else:
        return stripped.translate(_remove_last).strip()

In [ ]:
places = collections.Counter()
with AllEntries() as dataSet:
    for fn, year, entries in dataSet.getEntries():
        print(year)
        tmp = []
        for entry in entries:
            tmp.append(extractPlace(entry.fødested))
        places.update(tmp)
        del(tmp)

In [ ]:
places.most_common()

In [ ]:
differ = difflib.SequenceMatcher()

In [ ]:
differ.set_seq1("københavn")

In [ ]:
extractPlace("do. [rønne]")

In [ ]:
extractPlace("dito [kjøbenhavn]")

In [ ]:
extractPlace("kjøbenha[vn] !!")

In [ ]:
copenhagen_names = set()
for place, freq in places.most_common():
    differ.set_seq2(place)
    if differ.ratio() > 0.8:
        print(place, freq)
        copenhagen_names.add(place)

In [ ]:
len(places)

In [ ]:
with AllEntries() as dataSet:
    for fn, year, entries in dataSet.getEntries():
        print(year)
        lines = [commonHeader + "\n"]
        prev = None
        for entry in entries:
            if extractPlace(entry.fødested) in copenhagen_names:
                entry = entry._replace(fødested="københavn")
            lines.append(entry.toRow() + "\n")
        with fn.open("w", encoding="UTF-8") as fd:
            fd.writelines(lines)